# Exploring Variables

Original Variables:
- geostrophic velocity anomalies
- wind speed
- wind stress

Options to Explore:

- geostrophic velocity error bars from CMEMS gridded level 3 product
- cyclostrophic adjustments to the velocities due to the centrifugal term
- SST gradient 
- mixed layer depth 
- bathymetry
- drogue status 

In [2]:
import copernicusmarine
import xarray as xr